In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

imagen_alto = 128
imagen_ancho = 128
canales_de_color = 3

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(imagen_alto, imagen_ancho, canales_de_color)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

input_shape = tf.keras.Input(shape=(imagen_alto, imagen_ancho, canales_de_color))

c:\Users\achav\Documents\Osteoartritis\osteo\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Definir la arquitectura de la CNN
model = models.Sequential([
    (input_shape),  # Capa de entrada (128x128x1)
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 clases: Clase A y Clase B
]) 

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,267,778 (12.47 MB)

 Trainable params: 3,267,778 (12.47 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
import os
import cv2
import numpy as np
import pathlib

data_dir = pathlib.Path('dataset')

# Lista para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Clases disponibles
clases = ['class_A', 'class_B']

# Recorrer las carpetas de las clases
for clase in clases:
    for plate in os.listdir(data_dir / clase ):
        for file in os.listdir(data_dir / clase / plate / 'resized_128' ):
            # Leer la imagen
            imagen = cv2.imread(str(data_dir / clase / plate / 'resized_128' / file), cv2.IMREAD_UNCHANGED)
            
            # Normalizar la imagen
            imagen = imagen / 255.0
            # Agregar la imagen a la lista
            imagenes.append(imagen)
            # Agregar la etiqueta a la lista
            etiquetas.append(clases.index(clase))
            
# Convertir las listas de imágenes y etiquetas a arrays numpy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)


In [11]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y validación
imagenes_entrenamiento, imagenes_validacion, etiquetas_entrenamiento, etiquetas_validacion = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Convertir las listas de imágenes y etiquetas de entrenamiento a arrays numpy
imagenes_entrenamiento = np.array(imagenes_entrenamiento)
etiquetas_entrenamiento = np.array(etiquetas_entrenamiento)

# Convertir las listas de imágenes y etiquetas de validación a arrays numpy
imagenes_validacion = np.array(imagenes_validacion)
etiquetas_validacion = np.array(etiquetas_validacion)

In [12]:
from tensorflow.keras.callbacks import TensorBoard

num_epochs = 12
batch_size = 32

# Directorio donde se guardarán los registros para TensorBoard
# directorio_logs = "logs/"

# Callback de TensorBoard
# tensorboard_callback = TensorBoard(log_dir=directorio_logs, histogram_freq=1)

# Entrenar el modelo con el callback de TensorBoard
model.fit(imagenes_entrenamiento, etiquetas_entrenamiento,
          epochs=num_epochs,
          batch_size=batch_size,
)


Epoch 1/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 48s 238ms/step - accuracy: 0.8709 - loss: 0.4346
Epoch 2/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 48s 250ms/step - accuracy: 0.8929 - loss: 0.3430
Epoch 3/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 61s 320ms/step - accuracy: 0.8876 - loss: 0.3400
Epoch 4/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 49s 257ms/step - accuracy: 0.8911 - loss: 0.3046
Epoch 5/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 52s 273ms/step - accuracy: 0.8903 - loss: 0.2859
Epoch 6/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 54s 282ms/step - accuracy: 0.8982 - loss: 0.2748
Epoch 7/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 56s 295ms/step - accuracy: 0.8972 - loss: 0.2759
Epoch 8/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 56s 293ms/step - accuracy: 0.9008 - loss: 0.2681
Epoch 9/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 54s 286ms/step - accuracy: 0.9061 - loss: 0.2601
Epoch 10/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 83s 288ms/step - accuracy: 0.9006 - loss: 0.2715
Epoch 11/12
190/190 ━━━━━━━━━━━━━━━━━━━━ 52s 275ms/step - accuracy: 0.9096 - loss: 0.2513
Epoch 12/12
190/190

In [13]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(imagenes_validacion, etiquetas_validacion)
print('Precisión en datos de prueba:', test_acc)



48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.9010 - loss: 0.2669
Precisión en datos de prueba: 0.9063324332237244
